In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

def oku_ve_isleme(klasor, label, veri_tipi):
  """
  Belirtilen klasördeki .csv dosyalarını okur ve işler.

  Parametreler:
    klasor: Verilerin bulunduğu klasörün yolu.
    label: Verilerin etiketini ("Okra" veya "Normal").
    veri_tipi: Veri türü ("Intensity", "Absorbance", "Reflectance").

  Dönüş Değeri:
    İşlenmiş DataFrame.
  """
  dosyalar = os.listdir(klasor)
  tum_veri = pd.DataFrame()
  for dosya in dosyalar:
    dosya_yolu = os.path.join(klasor, dosya)
    veri = pd.read_csv(dosya_yolu)
    veri = veri.fillna(veri.mean())
    veri['Label'] = label
    if veri_tipi == "Intensity":
      veri.rename(columns={'Sample Signal (unitless)':'Intensity (Unitless)'}, inplace=True)
    else:
      veri.rename(columns={f'{veri_tipi} (unitless)':f'{veri_tipi} (AU)'}, inplace=True)
    tum_veri = pd.concat([tum_veri, veri], ignore_index=True)
  return tum_veri

def model_olustur(X, y):
  """
  SVM modeli oluşturur ve eğitir.

  Parametreler:
    X: Özellik matrisi.
    y: Etiket vektörü.

  Dönüş Değeri:
    Eğitilmiş SVM modeli.
  """
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
  model = SVC(kernel='linear', C=1)
  model.fit(X_train, y_train)
  return model

def degerlendir(model, X_test, y_test):
  """
  Modelin performansını değerlendirir.

  Parametreler:
    model: Eğitilmiş SVM modeli.
    X_test: Test seti özellik matrisi.
    y_test: Test seti etiket vektörü.

  Dönüş Değeri:
    Doğruluk oranı, karmaşıklık matrisi ve sınıflandırma raporu.
  """
  y_pred = model.predict(X_test)
  dogruluk_orani = accuracy_score(y_test, y_pred)
  print(f"\n### {veri_tipi} Verileri")
  print(f"Doğruluk Oranı: {dogruluk_orani:.4f}")
  print(confusion_matrix(y_test, y_pred))
  print(classification_report(y_test, y_pred))
  return dogruluk_orani, y_pred

# Farklı veri türleri için döngü
for veri_tipi in ["Intensity", "Absorbance", "Reflectance"]:
    # Verileri oku ve işle
    okra_klasoru = f"C:/Users/AliBakiTURKOZ/OneDrive/Masaüstü/{veri_tipi}_o"
    normal_klasoru = f"C:/Users/AliBakiTURKOZ/OneDrive/Masaüstü/{veri_tipi}_s"
    tum_veriler = oku_ve_isleme(okra_klasoru, 'Okra', veri_tipi)
    tum_veriler = pd.concat([tum_veriler, oku_ve_isleme(normal_klasoru, 'Normal', veri_tipi)], ignore_index=True)

    # Özellik matrisi ve etiket vektörü
    if veri_tipi == "Intensity":
        X = tum_veriler[['Wavelength (nm)', 'Intensity (Unitless)']]
    else:
        X = tum_veriler[['Wavelength (nm)', f'{veri_tipi} (AU)']]
        y = tum_veriler['Label']

        # Veri kümesini train ve test setlerine ayır
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Model oluştur ve eğit
        model = model_olustur(X_train, y_train)

        # Modelin performansını değerlendir
        dogruluk_orani, y_pred = degerlendir(model, X_test, y_test)
        
        # Düşük doğruluk oranlı örnekleri göster
        dusuk_dogruluk_orani_indeksleri = y_test[y_test != y_pred].index
        dusuk_dogruluk_orani_veriler = tum_veriler.iloc[dusuk_dogruluk_orani_indeksleri]
        print("\nDoğruluk Oranı Düşük Olan Örnekler:")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(dusuk_dogruluk_orani_veriler)




C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(



### Absorbance Verileri
Doğruluk Oranı: 0.5005
[[  0 911]
 [  0 913]]
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       911
        Okra       0.50      1.00      0.67       913

    accuracy                           0.50      1824
   macro avg       0.25      0.50      0.33      1824
weighted avg       0.25      0.50      0.33      1824


Doğruluk Oranı Düşük Olan Örnekler:
      Wavelength (nm)  Absorbance (AU)   Label
8225       969.143252         0.500871  Normal
4568       932.972595         0.564379  Normal
6022      1262.231332         0.906040  Normal
4855      1162.415303         0.673753  Normal
6049      1357.458757         1.033656  Normal
7222      1469.856129         1.659247  Normal
7048      1642.858971         1.662794  Normal
6715      1294.734032         0.834208  Normal
7807      1117.350441         0.483972  Normal
4890      1290.114340         0.694619  Normal
6889      1093.948010         0.603331  Normal
4

C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AliBakiTURKOZ\anaconda


### Reflectance Verileri
Doğruluk Oranı: 0.5005
[[  0 911]
 [  0 913]]
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       911
        Okra       0.50      1.00      0.67       913

    accuracy                           0.50      1824
   macro avg       0.25      0.50      0.33      1824
weighted avg       0.25      0.50      0.33      1824


Doğruluk Oranı Düşük Olan Örnekler:
      Wavelength (nm)  Reflectance (AU)   Label
8225       969.143252          0.315594  Normal
4568       932.972595          0.272660  Normal
6022      1262.231332          0.124154  Normal
4855      1162.415303          0.211956  Normal
6049      1357.458757          0.092543  Normal
7222      1469.856129          0.021916  Normal
7048      1642.858971          0.021737  Normal
6715      1294.734032          0.146485  Normal
7807      1117.350441          0.328116  Normal
4890      1290.114340          0.202014  Normal
6889      1093.948010          0.249

C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AliBakiTURKOZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
